# Flow SDK Inference Notebook

Deploy and serve large language models using Flow SDK with vLLM for high-performance inference.

This notebook covers:
- Quick model deployment
- OpenAI-compatible API serving
- Batch inference processing
- Performance optimization
- Cost monitoring

## Setup

First, let's install and configure the Flow SDK:

In [ ]:
# Install Flow SDK
!pip install flow-sdk --upgrade

# Import required libraries
import flow
from flow import TaskConfig
import json
import time
import requests
from typing import List, Dict

In [ ]:
# Initialize Flow client
flow_client = flow.Flow()

# Check authentication
print("✓ Flow SDK initialized")
print(f"API Endpoint: {flow_client.api_endpoint}")

## 1. Quick Model Deployment

Deploy a model with vLLM in under 2 minutes:

In [ ]:
# Deploy Mistral-7B with vLLM
inference_config = TaskConfig(
    name="mistral-inference-server",
    command="""
    pip install vllm
    
    python -m vllm.entrypoints.openai.api_server \
        --model mistralai/Mistral-7B-Instruct-v0.1 \
        --host 0.0.0.0 \
        --port 8000 \
        --max-model-len 8192
    """,
    instance_type="a100",  # Single A100 80GB
    ports=[8000],
    max_price_per_hour=10.00,  # Set your budget
    max_run_time_hours=24  # 24 hour deployment
)

# Launch the server
print("🚀 Launching inference server...")
inference_task = flow_client.run(inference_config)
print(f"Task ID: {inference_task.task_id}")
print(f"Status: {inference_task.status}")

In [ ]:
# Wait for server to be ready
print("⏳ Waiting for server to start...")

while True:
    task_info = flow_client.get_task(inference_task.task_id)
    if task_info.status == "running":
        print("✓ Server is running!")
        break
    elif task_info.status in ["failed", "cancelled"]:
        print(f"❌ Task {task_info.status}")
        print(task_info.logs())
        break
    time.sleep(10)

# Get endpoint information
if task_info.status == "running":
    endpoint = task_info.endpoints[0]
    print(f"\n🌐 API Endpoint: {endpoint}")
    print(f"💰 Current cost: ${task_info.total_cost:.3f}")

## 2. Test the Inference Server

Let's test our deployed model with some sample requests:

In [ ]:
# Create a helper function for API calls
def query_model(prompt: str, max_tokens: int = 100, temperature: float = 0.7):
    """Query the deployed model."""
    response = requests.post(
        f"{endpoint}/v1/completions",
        json={
            "model": "mistralai/Mistral-7B-Instruct-v0.1",
            "prompt": prompt,
            "max_tokens": max_tokens,
            "temperature": temperature
        }
    )
    return response.json()

# Test the model
test_prompt = "Explain quantum computing in simple terms:"
result = query_model(test_prompt, max_tokens=200)

print("🤖 Model Response:")
print("-" * 50)
print(result['choices'][0]['text'])

## 3. Batch Inference

Process multiple prompts efficiently:

In [ ]:
# Create batch inference configuration
batch_config = TaskConfig(
    name="batch-inference-job",
    command="""
    pip install vllm pandas tqdm
    
    python - << 'EOF'
import json
import pandas as pd
from vllm import LLM, SamplingParams
from tqdm import tqdm

# Sample prompts for batch processing
prompts = [
    "What are the benefits of renewable energy?",
    "Explain machine learning to a 10-year-old.",
    "What is the future of space exploration?",
    "How does cryptocurrency work?",
    "What are best practices for remote work?"
]

# Initialize model
print("Loading model...")
llm = LLM("mistralai/Mistral-7B-Instruct-v0.1", max_model_len=8192)

# Set generation parameters
params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=200
)

# Run batch inference
print(f"Processing {len(prompts)} prompts...")
outputs = llm.generate(prompts, params)

# Save results
results = []
for prompt, output in zip(prompts, outputs):
    results.append({
        "prompt": prompt,
        "response": output.outputs[0].text,
        "tokens": len(output.outputs[0].token_ids)
    })

# Save to file
with open("batch_results.json", "w") as f:
    json.dump(results, f, indent=2)

print(f"\nProcessed {len(results)} prompts")
print(f"Results saved to batch_results.json")

# Display sample
print("\nSample result:")
print(f"Prompt: {results[0]['prompt']}")
print(f"Response: {results[0]['response'][:200]}...")
EOF
    """,
    instance_type="a100",
    max_price_per_hour=10.00,
    max_run_time_hours=1,
    download_patterns=["batch_results.json"]
)

# Run batch job
print("🚀 Starting batch inference job...")
batch_task = flow_client.run(batch_config, wait=True)
print(f"\n✓ Batch job completed!")
print(f"Total cost: ${batch_task.total_cost:.3f}")

In [ ]:
# Download and display results
results_path = flow_client.download(batch_task.task_id, "batch_results.json")

with open(results_path, 'r') as f:
    batch_results = json.load(f)

# Display results in a nice format
for i, result in enumerate(batch_results[:3]):
    print(f"\n{'='*60}")
    print(f"Prompt {i+1}: {result['prompt']}")
    print(f"\nResponse: {result['response'][:300]}...")
    print(f"\nTokens generated: {result['tokens']}")

## 4. Multi-Model Serving

Deploy multiple models with a load balancer:

In [ ]:
# Create a multi-model server script
multi_model_script = """
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from vllm import LLM, SamplingParams
import uvicorn
import torch

app = FastAPI()

# Model registry
models = {
    "small": {
        "name": "microsoft/phi-2",
        "max_len": 2048,
        "description": "Fast 2.7B parameter model"
    },
    "medium": {
        "name": "mistralai/Mistral-7B-Instruct-v0.1",
        "max_len": 8192,
        "description": "Balanced 7B parameter model"
    }
}

# Load models
loaded_models = {}
for size, config in models.items():
    print(f"Loading {size} model: {config['name']}")
    loaded_models[size] = LLM(config['name'], max_model_len=config['max_len'])

class GenerateRequest(BaseModel):
    prompt: str
    model_size: str = "medium"
    max_tokens: int = 100
    temperature: float = 0.7

@app.get("/models")
async def list_models():
    return models

@app.post("/generate")
async def generate(request: GenerateRequest):
    if request.model_size not in loaded_models:
        raise HTTPException(400, f"Model size {request.model_size} not available")
    
    # Generate response
    params = SamplingParams(
        temperature=request.temperature,
        max_tokens=request.max_tokens
    )
    
    outputs = loaded_models[request.model_size].generate([request.prompt], params)
    
    return {
        "model": models[request.model_size]["name"],
        "response": outputs[0].outputs[0].text,
        "tokens": len(outputs[0].outputs[0].token_ids)
    }

@app.get("/health")
async def health():
    return {"status": "healthy", "models_loaded": list(loaded_models.keys())}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
"""

# Save the script
with open("/tmp/multi_model_server.py", "w") as f:
    f.write(multi_model_script)

print("✓ Multi-model server script created")

In [ ]:
# Deploy multi-model server
multi_model_config = TaskConfig(
    name="multi-model-server",
    command="""
    pip install vllm fastapi uvicorn
    python /workspace/multi_model_server.py
    """,
    instance_type="2xa100",  # 2x A100 for multiple models
    ports=[8000],
    upload_files={"/tmp/multi_model_server.py": "multi_model_server.py"},
    max_price_per_hour=20.00,
    max_run_time_hours=12
)

print("🚀 Deploying multi-model server...")
multi_task = flow_client.run(multi_model_config)
print(f"Task ID: {multi_task.task_id}")

## 5. Performance Optimization

Optimize inference for better throughput and lower latency:

In [ ]:
# Benchmark different configurations
benchmark_config = TaskConfig(
    name="inference-benchmark",
    command="""
    pip install vllm pandas matplotlib
    
    python - << 'EOF'
import time
import json
from vllm import LLM, SamplingParams
import matplotlib.pyplot as plt

# Test configurations
configs = [
    {"tensor_parallel_size": 1, "max_num_seqs": 256},
    {"tensor_parallel_size": 1, "max_num_seqs": 512},
    {"tensor_parallel_size": 1, "max_num_seqs": 1024},
]

results = []

for config in configs:
    print(f"\nTesting config: {config}")
    
    # Initialize model with config
    llm = LLM(
        "mistralai/Mistral-7B-v0.1",
        tensor_parallel_size=config["tensor_parallel_size"],
        max_num_seqs=config["max_num_seqs"]
    )
    
    # Create test prompts
    test_prompts = ["Generate a random sentence." for _ in range(100)]
    
    # Benchmark
    start_time = time.time()
    outputs = llm.generate(test_prompts, SamplingParams(max_tokens=50))
    end_time = time.time()
    
    # Calculate metrics
    total_time = end_time - start_time
    throughput = len(test_prompts) / total_time
    avg_latency = total_time / len(test_prompts)
    
    result = {
        "config": config,
        "throughput_req_per_sec": throughput,
        "avg_latency_sec": avg_latency,
        "total_time": total_time
    }
    results.append(result)
    
    print(f"Throughput: {throughput:.2f} req/s")
    print(f"Avg latency: {avg_latency:.3f} s")

# Save results
with open("benchmark_results.json", "w") as f:
    json.dump(results, f, indent=2)

# Plot results
plt.figure(figsize=(10, 6))
max_seqs = [r["config"]["max_num_seqs"] for r in results]
throughputs = [r["throughput_req_per_sec"] for r in results]

plt.bar(range(len(max_seqs)), throughputs)
plt.xlabel("Max Sequences Configuration")
plt.ylabel("Throughput (req/s)")
plt.title("vLLM Throughput vs Configuration")
plt.xticks(range(len(max_seqs)), [str(s) for s in max_seqs])
plt.savefig("benchmark_plot.png")
plt.close()

print("\nBenchmark complete! Results saved.")
EOF
    """,
    instance_type="a100",
    max_price_per_hour=10.00,
    max_run_time_hours=1,
    download_patterns=["benchmark_results.json", "benchmark_plot.png"]
)

print("⚡ Running performance benchmark...")
benchmark_task = flow_client.run(benchmark_config, wait=True)
print("✓ Benchmark complete!")

## 6. Cost Monitoring & Optimization

Monitor and optimize inference costs:

In [ ]:
# Get current running tasks
running_tasks = flow_client.list_tasks(status="running")

print("💰 Current Running Inference Tasks:")
print("-" * 60)
print(f"{'Task Name':<30} {'Instance':<15} {'Cost/Hour':<10} {'Total':<10}")
print("-" * 60)

total_hourly = 0
for task in running_tasks:
    if "inference" in task.name or "server" in task.name:
        hourly_cost = getattr(task, 'cost_per_hour', 0)
        total_cost = getattr(task, 'total_cost', 0)
        total_hourly += hourly_cost
        
        print(f"{task.name:<30} {task.instance_type:<15} ${hourly_cost:<9.2f} ${total_cost:<9.2f}")

print("-" * 60)
print(f"{'Total Hourly Cost:':<30} {'':<15} ${total_hourly:<9.2f}")
print(f"\n📊 Daily projection: ${total_hourly * 24:.2f}")
print(f"📊 Monthly projection: ${total_hourly * 24 * 30:.2f}")

In [ ]:
# Cost optimization recommendations
def get_instance_recommendation(model_size_gb: float, concurrent_requests: int) -> str:
    """Recommend optimal instance type based on workload."""
    
    # Model memory requirement (with overhead)
    memory_needed = model_size_gb * 2.5
    
    if memory_needed <= 80 and concurrent_requests <= 50:
        return "a100"  # Single GPU
    elif memory_needed <= 160 or concurrent_requests <= 100:
        return "2xa100"
    elif memory_needed <= 320 or concurrent_requests <= 200:
        return "4xa100"
    else:
        return "8xa100"

# Example recommendations
workloads = [
    {"model": "Llama-2-7B", "size_gb": 13, "requests": 20},
    {"model": "Llama-2-13B", "size_gb": 26, "requests": 50},
    {"model": "Llama-2-70B", "size_gb": 140, "requests": 10},
    {"model": "Mixtral-8x7B", "size_gb": 90, "requests": 30},
]

print("🎯 Instance Recommendations:")
print("-" * 70)
print(f"{'Model':<20} {'Size':<10} {'Requests/s':<15} {'Recommended':<15}")
print("-" * 70)

for w in workloads:
    rec = get_instance_recommendation(w['size_gb'], w['requests'])
    print(f"{w['model']:<20} {w['size_gb']:>3} GB      {w['requests']:<15} {rec:<15}")

print("\n💡 Note: Mithril uses dynamic pricing. Use 'flow instances' to check current rates.")

## 7. Production Deployment Pattern

Complete production deployment with monitoring and auto-scaling:

In [ ]:
# Production deployment configuration
production_config = TaskConfig(
    name="production-inference-server",
    command="""
    # Install dependencies
    pip install vllm prometheus-client
    
    # Create monitoring wrapper
    cat > server_with_monitoring.py << 'SCRIPT'
import os
from prometheus_client import start_http_server, Counter, Histogram, Gauge
import time
import threading

# Metrics
request_count = Counter('inference_requests_total', 'Total inference requests')
request_latency = Histogram('inference_request_latency_seconds', 'Request latency')
active_requests = Gauge('inference_active_requests', 'Active requests')
model_loaded = Gauge('model_loaded', 'Model load status')

# Start metrics server
start_http_server(9090)
print("Metrics server started on port 9090")

# Start vLLM
model_loaded.set(1)
os.system("""
python -m vllm.entrypoints.openai.api_server \
    --model mistralai/Mistral-7B-Instruct-v0.1 \
    --host 0.0.0.0 \
    --port 8000 \
    --max-model-len 8192 \
    --gpu-memory-utilization 0.9
""")
SCRIPT
    
    # Run with monitoring
    python server_with_monitoring.py
    """,
    instance_type="a100",
    ports=[8000, 9090],  # API and metrics ports
    max_price_per_hour=15.00,
    max_run_time_hours=168,  # 1 week
    
    # Auto-restart on failure
    retry_on_failure=True,
    max_retries=3,
    
    # Environment variables
    environment={
        "VLLM_ATTENTION_BACKEND": "FLASHINFER",
        "CUDA_VISIBLE_DEVICES": "0"
    }
)

print("🚀 Deploying production inference server...")
prod_task = flow_client.run(production_config)
print(f"Production Task ID: {prod_task.task_id}")

## 8. Cleanup

Don't forget to stop your inference servers when done:

In [ ]:
# List all running inference tasks
inference_tasks = [
    task for task in flow_client.list_tasks(status="running") 
    if "inference" in task.name.lower() or "server" in task.name.lower()
]

print(f"Found {len(inference_tasks)} running inference tasks:")
for task in inference_tasks:
    print(f"  - {task.name} (ID: {task.task_id})")

# Uncomment to stop all inference tasks
# for task in inference_tasks:
#     print(f"Stopping {task.name}...")
#     flow_client.cancel(task.task_id)
#     print("✓ Stopped")

## Summary

In this notebook, you learned how to:

1. **Deploy models quickly** with vLLM
2. **Run batch inference** for processing large datasets
3. **Serve multiple models** from a single endpoint
4. **Optimize performance** with benchmarking
5. **Monitor costs** and get instance recommendations
6. **Deploy production-ready** inference servers

### Next Steps

- Explore the [Training Notebook](training.ipynb) for model training
- Check out [Fine-tuning Notebook](fine-tuning.ipynb) for customizing models
- Read the [Production Guide](../../guides/production-inference.md) for scaling

### Key Takeaways

- Start with single GPU (`a100`) and scale up as needed
- Always set budget limits with `max_price_per_hour`
- Use vLLM for best inference performance
- Monitor costs and optimize instance selection
- Mithril uses dynamic pricing - check current rates with `flow instances`